In [74]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [75]:
df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [76]:
# tst = pd.read_csv('submission_20200430_5.csv')
# sub = pd.read_csv('submission_20200430_5.csv')
# for col in sub.columns[1:]:
#     sub.loc[sub[col]<0.1,col] = 0
# sub.to_csv('submission_20200430_6.csv',index=False)

In [213]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm


class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame, tst):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()
#         train_id = train_id
        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices
        self.tst = tst
        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        scale = np.where(scale != 0 , scale, 1)
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            all_scores.append(lv_scores.sum())

        return np.mean(all_scores)
    
    
class WRMSSEForLightGBM(WRMSSEEvaluator):

    def feval(self, preds, dtrain):
#         print(preds.shape, self.tst.shape)
#         tst= self.df[self.df['day'].isin(valid_target_columns)]
#         tst['id'] = train_id.loc[tst.index]
        tmp = self.tst.copy()
        tmp['preds'] = preds
        tmp=  tmp.set_index(['id',"day"]).unstack()["preds"].reset_index()
        tmp.tst =  tmp.fillna(0)

        val = pd.DataFrame()
        val['id'] = self.train_df['id']
        pred = pd.merge(val, tmp, how = 'left')
        pred = pred.fillna(0)
        pred = pred.loc[:,self.valid_target_columns ]
#         cv_scores.append(evaluator.score(pred))
#         preds = preds.reshape(self.valid_df[self.valid_target_columns].shape)
        score = self.score(pred)
        return 'WRMSSE', score, False

In [78]:
# folds = 3
# cols = ["d_%s"%x for x in range(1,1914)]
# n= int(np.round(len(cols)/folds))
# for i in range(3):
#     d_valid = cols[n*(i):n*(i+1)]
#     d_train = [x for x in cols if x not in d_valid]
#     tmp = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
#     tmp_train = pd.concat([tmp.iloc[:,:6],tmp.loc[:, d_train]], axis =1)
#     tmp_valid = df_train.loc[:, d_valid]

    
    

,d_1277,d_1278,d_1279,d_1280,d_1281,d_1282,d_1283,d_1284,d_1285,d_1286,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,44,100,42,44,57,10,34,38,67,82,...,79,96,79,72,72,3,79,65,1,46
1,15,53,83,5,31,95,11,24,29,5,...,48,47,43,10,95,6,58,2,63,66
2,84,44,22,18,43,58,94,36,32,55,...,37,5,91,69,61,50,19,35,66,66
3,8,60,61,73,26,25,80,58,53,43,...,70,77,60,42,37,87,48,81,89,96
4,63,56,16,60,24,6,81,70,59,23,...,62,61,39,77,69,19,42,87,24,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,90,97,82,70,25,36,36,83,37,25,...,49,55,47,45,4,21,5,17,56,67
30486,66,58,20,95,21,49,24,69,88,32,...,82,65,9,39,70,65,67,61,92,43
30487,2,72,37,93,18,3,69,80,82,9,...,19,99,77,98,3,30,32,61,7,17
30488,33,24,20,98,15,8,7,67,32,62,...,50,32,12,91,13,8,54,6,44,64


In [79]:
# tmp_train = pd.concat([df_train.iloc[:,:6],df_train.loc[:, d_train]], axis =1)
# tmp_valid = df_train.loc[:, d_valid]

# valid_preds = tmp_valid.copy() + np.random.randint(100, size=tmp_valid.shape)

# evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
# evaluator.score(valid_preds)


In [80]:
# train_fold_df

In [81]:
# # train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
# train_fold_df = df_train.iloc[:, :-28]
# valid_fold_df = df_train.iloc[:, -28:]
# valid_preds = valid_fold_df.copy() + np.random.randint(100, size=valid_fold_df.shape)

# evaluator = WRMSSEEvaluator(train_fold_df, valid_fold_df, calendar, price)
# evaluator.score(valid_preds)

In [82]:
# evaluator.lv12_weight.prod(axis=1)

In [83]:
# for i in range(0,12):
#     print(getattr(evaluator, f'lv{i + 1}_weight'))
    

In [84]:
# for i in range(0,12):
#     print(getattr(evaluator, f'lv{i + 1}_scale'))


In [85]:
# for i in range(0,12):
#     print(getattr(evaluator, f'lv{i + 1}_valid_df'))
    

In [86]:
# evaluator.get_weight_df()

In [87]:

    
# evaluator = WRMSSEForLightGBM(train_fold_df, valid_fold_df, calendar, prices)
# model = lgb.train(params, dtrain,
#                   num_boost_round=10000,
#                   valid_sets=dvalid,
#                   feval=evaluator.feval,
# #                   early_stopping_rounds=200)

In [88]:
# # startpoints 찾아서 이전 데이터 지우기 + last sales를 찾기
startpoints = np.zeros(df_train.shape[0])
lastpoints = np.zeros(df_train.shape[0])
for idx in tqdm(range(df_train.shape[0])):
    startpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].min().astype(int)
    lastpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].max().astype(int)
start_dict = dict(zip(df_train['id'], startpoints))
last_dict = dict(zip(df_train['id'], lastpoints))

In [89]:
drop_cols = []
cat_cols = []
drop_cols += ['date','d','id']
tr_last = 1913
# F_1~28 만들기  1914~1941 까지 
for i in range(tr_last+1, tr_last+1+28):   df_train['d_%s'%i] = 0

# # Unpivot
df_train = pd.melt(df_train, id_vars=df_train.columns[:6], value_vars=df_train.columns[6:],
       var_name = 'day', value_name = 'volume')



In [90]:
df_train = pd.merge(df_train, calendar, left_on = 'day', right_on ='d')
# snap 합치기
snap = np.zeros(df_train.shape[0])
snap[df_train[(df_train['state_id']=='CA')&(df_train['snap_CA']==1)].index] +=1
snap[df_train[(df_train['state_id']=='TX')&(df_train['snap_TX']==1)].index] +=1
snap[df_train[(df_train['state_id']=='WI')&(df_train['snap_WI']==1)].index] +=1
df_train['snap'] = snap
drop_cols += ['snap_CA','snap_TX','snap_WI']


cat_cols += [ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
#               'wday', 'month', 'year', # 이게 크리티컬?
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap']


# ????

In [91]:
# Sell price
df_train.head()
df_train = pd.merge(df_train, price)

# # Start point 찾기 ::  0.1% 데이터를 날릴  수 있다. 

df_train['startpoint'] = df_train['id'].map(start_dict).astype(int)#.astype(str)
df_train['startpoints'] = df_train['day'].str.slice(start=2).astype(int) >=df_train['startpoint']
print(df_train['startpoints'].value_counts())
df_train = df_train[df_train['startpoints']]
print(df_train.shape)
df_train.drop(['startpoint','startpoints'],axis =1, inplace= True)

True     46816555
False       65122
Name: startpoints, dtype: int64
(46816555, 26)


In [92]:
df_train['lastpoint'] = df_train['id'].map(last_dict).astype(int)#.astype(str)
df_train['from_lastpoint'] = df_train['day'].str.slice(start=2).astype(int) - df_train['lastpoint']

df_train.drop(['lastpoint'],axis =1, inplace= True)

In [93]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,volume,date,wm_yr_wk,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap,sell_price,from_lastpoint
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1911
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1910
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1909
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,1,1,0,1.0,0.46,-1908
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,1,0,1,1.0,0.46,-1907


In [94]:
df_train

,id,item_id,dept_id,cat_id,store_id,state_id,day,volume,date,wm_yr_wk,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap,sell_price,from_lastpoint
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1911
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1910
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1909
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,1,1,0,1.0,0.46,-1908
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,1,0,1,1.0,0.46,-1907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,3.98,30
46881673,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1940,0,2016-05-21,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.28,28
46881674,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.28,29
46881675,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1940,0,2016-05-21,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.00,43


In [12]:
# df_train['rcount_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# df_train[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0 else 0 )#.fllna(0)


In [13]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

# df_train['volume_1'] = df_train[['id','volume']].groupby("id")['volume'].shift(1)
# df_train['volume_2'] = df_train[['id','volume']].groupby("id")['volume'].shift(2)
# df_train['volume_3'] = df_train[['id','volume']].groupby("id")['volume'].shift(3)

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())

print("std")
df_train['rstd_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).std())
df_train['rstd_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).std())
df_train['rstd_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).std())

df_train['rstd_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).std())
df_train['rstd_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).std())
df_train['rstd_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).std())

print("max")
df_train['rmax_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).max())
df_train['rmax_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).max())
df_train['rmax_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).max())

df_train['rmax_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).max())
df_train['rmax_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).max())
df_train['rmax_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).max())

# print("min")
# df_train['rmin_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).min())
# df_train['rmin_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).min())
# df_train['rmin_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).min())

# df_train['rmin_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).min())
# df_train['rmin_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).min())
# df_train['rmin_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).min())

# print("count")
# df_train['rcount_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# df_train['rcount_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# df_train['rcount_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

# df_train['rcount_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).count() if x>0 else 0).fllna(0)
# df_train['rcount_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).count() if x>0 else 0).fllna(0)
# df_train['rcount_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).count() if x>0 else 0).fllna(0)

# full_df['rcnt_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# full_df['rcnt_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# full_df['rcnt_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)


mean
std
max
min
Wall time: 13min 13s


In [95]:
df_train['date'] =  pd.to_datetime(df_train["date"])

In [96]:
df_train['week'] = getattr(df_train["date"].dt, "weekofyear").astype("int16")
# df_train['quarter'] = getattr(df_train["date"].dt,"quarter").astype("int16")
df_train['mday'] = getattr(df_train["date"].dt, "day").astype("int16")


In [97]:
cols =['event_name_1','event_type_1','event_name_2','event_type_2']
df_train[cols]= df_train[cols].fillna('NaN')
print(df_train.shape)


(46816555, 27)


In [98]:
gc.collect()

153

In [99]:
df_train.shape

(46816555, 27)

In [102]:
df_train = df_train[~df_train['rmean_28_50'].isna()]
# dropna 하면 뻥난다.

In [101]:
for col in df_train.columns:
    print(col, df_train[col].isnull().sum())
# df_train.isnull().sum()

id 0
item_id 0
dept_id 0
cat_id 0
store_id 0
state_id 0
day 0
volume 0
date 0
wm_yr_wk 0
weekday 0
wday 0
month 0
year 0
d 0
event_name_1 0
event_type_1 0
event_name_2 0
event_type_2 0
snap_CA 0
snap_TX 0
snap_WI 0
snap 0
sell_price 0
from_lastpoint 0
week 0
mday 0


In [ ]:
# df_train.dropna()#inplace =True

In [103]:
df_train.shape

(46816555, 27)

In [104]:
drop_cols += ["wm_yr_wk", "weekday"]  ## 이게 문제?
drop_cols

['date', 'd', 'id', 'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk', 'weekday']

In [105]:
df_train.head()
tr_last = 1913
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
train_id = df_train['id']
df_test_id = df_train[df_train['day'].isin(testday)]['id']

In [106]:
df_train= df_train.drop(drop_cols,axis =1 )
# df_test =df_test.drop(drop_cols,axis =1)

In [107]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,from_lastpoint,week,mday
0,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,1,1,2011,NaN,NaN,NaN,NaN,0.0,0.46,-1911,4,29
1,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2,1,2011,NaN,NaN,NaN,NaN,0.0,0.46,-1910,4,30
2,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,3,1,2011,NaN,NaN,NaN,NaN,0.0,0.46,-1909,5,31
3,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,4,2,2011,NaN,NaN,NaN,NaN,1.0,0.46,-1908,5,1
4,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,5,2,2011,NaN,NaN,NaN,NaN,1.0,0.46,-1907,5,2


In [108]:
# Encoding
for col in tqdm(cat_cols) :  # encoding -1이 문제?
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col]).astype(np.int8)

    gc.collect()

In [109]:
# Test 분리하기
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
# df_test = df_train.copy()
df_train = df_train[~df_train['day'].isin(testday)]
# train_col = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day', 'volume', 'wday', 'month', 'year',
#  'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap', 'sell_price', 'volume_7', 'volume_28',
#  'rmean_7_7', 'rmean_7_28', 'rmean_7_50', 'rmean_28_7', 'rmean_28_28', 'rmean_28_50', 'week', 'quarter', 'mday']
# df_train = df_train.loc[:,train_col]

In [111]:
# df_test.head()

In [ ]:
# %%time

# from sklearn.model_selection import train_test_split

# x_train , x_valid = train_test_split(df_train, test_size =0.15, random_state = 99)
# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

In [ ]:
df_train.head()

In [217]:
# train_id

In [227]:
params = {
        "objective" : "poisson",
#         "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 1,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],#"None",#
    'verbosity': 1,
    'num_iterations' : 10,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
        'n_jobs' :1 
}

In [ ]:
# # tmp.loc[:, d_train]
# tmp_train = pd.concat([tmp.iloc[:,:6],tmp.loc[:, d_train]], axis =1)
# # tmp_valid = df_train.loc[:, d_valid]

In [230]:
%%time
# New Cross validation
import pickle
cv_scores = []
folds = 3
cols = ["d_%s"%x for x in range(1,1914)]
n= int(np.round(len(cols)/folds))
for idx in range(folds):
#     print(i)
    if idx ==0 : continue
    if idx ==1 : continue

    print(idx)
    # Set Evaluator
    d_valid = cols[n*(idx):n*(idx+1)]
    d_train = [x for x in cols if x not in d_valid]
    tmp = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
    tmp_train = pd.concat([tmp.iloc[:,:6],tmp.loc[:, d_train]], axis =1)
    tmp_valid = tmp.loc[:, d_valid]
#     evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
    tst= df_train[df_train['day'].isin(d_valid)]
    tst['id'] = train_id.loc[tst.index]
    evaluator = WRMSSEForLightGBM(tmp_train, tmp_valid, calendar, price,tst)

    del tmp 
    gc.collect()
    
    # Set train, valid
    x_train  = df_train[df_train['day'].isin(d_train)].drop(['day','volume'], axis =1)
    y_train = df_train[df_train['day'].isin(d_train)]['volume']
    x_valid  = df_train[df_train['day'].isin(d_valid)].drop(['day','volume'], axis =1)
    y_valid = df_train[df_train['day'].isin(d_valid)]['volume']
    print("set train, valid")
    # Modeling
    lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
    lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
    gbm = lgb.train(params, lgb_train, valid_sets=(lgb_eval),#(lgb_train, lgb_eval),
                    feval= evaluator.feval,# Custom Loss 사용
#                     callbacks = [lgb.early_stopping(10, first_metric_only=True)],# Metric First 체크 -earyly Stopping 에
                    early_stopping_rounds= 10,  # rmse 를 기준으로 하고 -- 분석한 후에 WRMSSE만 넣어서 할것
#                 learning_rates=lambda iter: (1 / (1 + decay_rate * iter)) * params['learning_rate'])
                    verbose_eval=1) 

    pickle.dump(gbm,open( "20200502_model_%s_r1_test.pkl"%idx, "wb" ))
    
#     evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
    
    preds= gbm.predict(x_valid)
    tst= df_train[df_train['day'].isin(d_valid)]
    tst['id'] = train_id.loc[tst.index]
    tst['preds'] = preds
    tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
    tst = tst.fillna(0)

    val = pd.DataFrame()
    val['id'] = tmp_train['id']
    pred = pd.merge(val,tst, how = 'left')
    pred = pred.fillna(0)
    pred = pred.loc[:,d_valid]
    cv_scores.append(evaluator.score(pred))
    print("idx","WRMSSE :", evaluator.score(pred))
    
    
#     del gbm,tmp_train, tmp_valid, evaluator
#     gc.collect()
    
    

    
    

2


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice.



set train, valid
(19105280,) (19105280, 20)


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:109: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


[1]	valid_0's WRMSSE: 3.23408e+06
Training until validation scores don't improve for 10 rounds
(19105280,) (19105280, 20)
[2]	valid_0's WRMSSE: 1.96828e+06
(19105280,) (19105280, 20)
[3]	valid_0's WRMSSE: 1.19791e+06
(19105280,) (19105280, 20)
[4]	valid_0's WRMSSE: 729054
(19105280,) (19105280, 20)
[5]	valid_0's WRMSSE: 443706
(19105280,) (19105280, 20)
[6]	valid_0's WRMSSE: 270043
(19105280,) (19105280, 20)
[7]	valid_0's WRMSSE: 5.88887e+26
(19105280,) (19105280, 20)
[8]	valid_0's WRMSSE: 3.55041e+135
(19105280,) (19105280, 20)
[9]	valid_0's WRMSSE: 3.55041e+135
(19105280,) (19105280, 20)
[10]	valid_0's WRMSSE: 3.55041e+135
Did not meet early stopping. Best iteration is:
[6]	valid_0's WRMSSE: 270043
Evaluated only: WRMSSE


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


idx WRMSSE : 270042.70169934334
Wall time: 13min 14s


In [215]:
%%time
# New Cross validation
import pickle
cv_scores = []
folds = 3
cols = ["d_%s"%x for x in range(1,1914)]
n= int(np.round(len(cols)/folds))
for idx in range(folds):
#     print(i)
    if idx ==0 : continue
    if idx ==1 : continue

    print(idx)
    # Set Evaluator
    d_valid = cols[n*(idx):n*(idx+1)]
    d_train = [x for x in cols if x not in d_valid]
    tmp = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
    tmp_train = pd.concat([tmp.iloc[:,:6],tmp.loc[:, d_train]], axis =1)
    tmp_valid = tmp.loc[:, d_valid]
#     evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
    tst= df_train[df_train['day'].isin(d_valid)]
    tst['id'] = train_id.loc[tst.index]
    evaluator = WRMSSEForLightGBM(tmp_train, tmp_valid, calendar, price,tst)
# model = lgb.train(params, dtrain,
#                   num_boost_round=10000,
#                   valid_sets=dvalid,
#                   feval=evaluator.feval,
#                   early_stopping_rounds=200)
    del tmp 
    gc.collect()
    
    # Set train, valid
    x_train  = df_train[df_train['day'].isin(d_train)].drop(['day','volume'], axis =1)
    y_train = df_train[df_train['day'].isin(d_train)]['volume']
    x_valid  = df_train[df_train['day'].isin(d_valid)].drop(['day','volume'], axis =1)
    y_valid = df_train[df_train['day'].isin(d_valid)]['volume']
    print("set train, valid")
    # Modeling
    lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
    lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
    gbm = lgb.train(params, lgb_train, valid_sets=(lgb_eval),#(lgb_train, lgb_eval),
#                     feval=evaluator.feval,
#                     early_stopping_rounds= 10,
                    verbose_eval=1) 
#     del evaluator
#     gc.collect()
    pickle.dump(gbm,open( "20200502_model_%s_r1_test.pkl"%idx, "wb" ))
    
#     evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
    
    preds= gbm.predict(x_valid)
    tst= df_train[df_train['day'].isin(d_valid)]
    tst['id'] = train_id.loc[tst.index]
    tst['preds'] = preds
    tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
    tst = tst.fillna(0)

    val = pd.DataFrame()
    val['id'] = tmp_train['id']
    pred = pd.merge(val,tst, how = 'left')
    pred = pred.fillna(0)
    pred = pred.loc[:,d_valid]
    cv_scores.append(evaluator.score(pred))
    print("idx","WRMSSE :", evaluator.score(pred))
    
    
#     del gbm,tmp_train, tmp_valid, evaluator
#     gc.collect()
    
    

    
    

2


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice.



set train, valid
[1]	valid_0's rmse: 8.58595e+07
[2]	valid_0's rmse: 5.22545e+07
[3]	valid_0's rmse: 3.18024e+07
[4]	valid_0's rmse: 1.93551e+07
[5]	valid_0's rmse: 1.17796e+07
[6]	valid_0's rmse: 7.16916e+06
[7]	valid_0's rmse: 1.0538e+28
[8]	valid_0's rmse: 5.96353e+135
[9]	valid_0's rmse: 5.96353e+135
[10]	valid_0's rmse: 5.96353e+135


KeyboardInterrupt: 

In [209]:
x_train.shape

(26857555, 17)

In [211]:
x_valid.shape

(19105280, 17)

In [180]:
#전부를 대상으로 해야한다.   ---- >>>  30490 row를 만들어내야한다. 
# train_id 특정 위치에 예측값을 넣도록 바꾸자
# np.zero(tmp_valid.shape) --> 0으로 넣고
# 그 자리를 채우도록 바꾸자 


# tst= df_train[df_train['day'].isin(d_valid)]
# tst['id'] = train_id.loc[tst.index]
# tst['preds'] = preds
# tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
# tst = tst.fillna(0)

# val = pd.DataFrame()
# val['id'] = tmp_train['id']
# pred = pd.merge(val,tst, how = 'left')
# pred = k.fillna(0)
# pred
# tst = tst.loc[:,d_valid]
# tst = tst.drop(['id'], axis =1)
# tst

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,id,d_1,d_10,d_100,d_101,d_102,d_103,d_104,d_105,d_106,...,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986,...,0.584033,0.584033,0.653986,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.000000,0.469613,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670,...,0.469613,0.469613,1.235670,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670
30486,FOODS_3_824_WI_3_validation,0.000000,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,...,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613
30487,FOODS_3_825_WI_3_validation,1.310041,1.193594,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041,...,1.193594,1.193594,1.310041,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041
30488,FOODS_3_826_WI_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [186]:
preds.shape

(10892478,)

In [190]:
pred

,id,d_1,d_10,d_100,d_101,d_102,d_103,d_104,d_105,d_106,...,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986,...,0.584033,0.584033,0.653986,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.000000,0.469613,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670,...,0.469613,0.469613,1.235670,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670
30486,FOODS_3_824_WI_3_validation,0.000000,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,...,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613
30487,FOODS_3_825_WI_3_validation,1.310041,1.193594,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041,...,1.193594,1.193594,1.310041,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041
30488,FOODS_3_826_WI_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [189]:
tst= df_train[df_train['day'].isin(d_valid)]
tst['id'] = train_id.loc[tst.index]
tst['preds'] = preds
tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
tst = tst.fillna(0)

val = pd.DataFrame()
val['id'] = tmp_train['id']
pred = pd.merge(val,tst, how = 'left')p
pred = k.fillna(0)
pred = pred.loc[:,d_valid]
cv_scores.append(evaluator.score(pred))
print("WRMSSE :", evaluator.score(pred))

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "None of [Index(['d_639', 'd_640', 'd_641', 'd_642', 'd_643', 'd_644', 'd_645', 'd_646',\n       'd_647', 'd_648',\n       ...\n       'd_1267', 'd_1268', 'd_1269', 'd_1270', 'd_1271', 'd_1272', 'd_1273',\n       'd_1274', 'd_1275', 'd_1276'],\n      dtype='object', length=638)] are in the [columns]"

In [179]:
# tmp_valid['id'] = train_id
# tmp_valid


,id,d_1,d_10,d_100,d_101,d_102,d_103,d_104,d_105,d_106,...,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986,...,0.584033,0.584033,0.653986,0.653986,0.584033,0.584033,0.584033,0.584033,0.584033,0.653986
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.000000,0.469613,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670,...,0.469613,0.469613,1.235670,1.235670,1.125834,1.125834,1.125834,1.125834,1.125834,1.235670
30486,FOODS_3_824_WI_3_validation,0.000000,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,...,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613,0.469613
30487,FOODS_3_825_WI_3_validation,1.310041,1.193594,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041,...,1.193594,1.193594,1.310041,1.310041,1.193594,1.193594,1.193594,1.193594,1.193594,1.310041
30488,FOODS_3_826_WI_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [172]:
tst

day,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_629,d_630,d_631,d_632,d_633,d_634,d_635,d_636,d_637,d_638
0,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,...,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571
1,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,...,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571
2,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,...,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628
3,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,...,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709
4,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,...,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21335,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,...,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816
21336,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485
21337,0.000000,0.000000,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485
21338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485


In [164]:
tst = tst.loc[:,d_valid]
tst

day,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_629,d_630,d_631,d_632,d_633,d_634,d_635,d_636,d_637,d_638
0,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,...,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571
1,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,...,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571,0.813571
2,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,...,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628,0.968628
3,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,...,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709
4,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,...,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709,0.713709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21335,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,...,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816,0.511816
21336,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485
21337,0.000000,0.000000,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485
21338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485,0.454485


In [162]:
tst.shape

(21340, 638)

In [163]:
tmp_valid.shape

(30490, 638)

In [161]:
# tst= df_train[df_train['day'].isin(d_valid)]
# tst['id'] = train_id.loc[tst.index]
# tst['preds'] = preds
# tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
# tst = tst.fillna(0)
# tst = tst.loc[:,d_valid]
# # tst = tst.drop(['id'], axis =1)
# tst
evaluator.score(gbm.predict(x_valid))

AssertionError: 

In [152]:
tst

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,from_lastpoint,week,mday,preds
0,-92,3,1,0,0,d_1,12,1,1,2011,18,1,3,1,0,0.46,-1911,4,29,6.735440
1,-92,3,1,0,0,d_2,15,2,1,2011,18,1,3,1,0,0.46,-1910,4,30,6.735440
2,-92,3,1,0,0,d_3,0,3,1,2011,18,1,3,1,0,0.46,-1909,5,31,5.762981
3,-92,3,1,0,0,d_4,0,4,2,2011,18,1,3,1,1,0.46,-1908,5,1,5.762981
4,-92,3,1,0,0,d_5,0,5,2,2011,18,1,3,1,1,0.46,-1907,5,2,5.762981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11054771,-107,2,0,9,2,d_638,1,1,10,2012,18,1,3,1,0,1.98,-1274,43,27,0.469613
11054778,-105,2,0,9,2,d_638,0,1,10,2012,18,1,3,1,0,3.98,-1274,43,27,1.310041
11054785,-104,2,0,9,2,d_638,1,1,10,2012,18,1,3,1,0,2.68,-1274,43,27,0.974031
11054792,-103,2,0,9,2,d_638,1,1,10,2012,18,1,3,1,0,2.68,-1273,43,27,0.974031


In [137]:
preds = gbm.predict(x_valid)
preds = preds.reshape(tmp_valid.shape)
# score = self.score(preds)
preds

ValueError: cannot reshape array of size 10892478 into shape (30490,638)

In [140]:
x_valid

,item_id,dept_id,cat_id,store_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,from_lastpoint,week,mday
0,-92,3,1,0,0,1,1,2011,18,1,3,1,0,0.46,-1911,4,29
1,-92,3,1,0,0,2,1,2011,18,1,3,1,0,0.46,-1910,4,30
2,-92,3,1,0,0,3,1,2011,18,1,3,1,0,0.46,-1909,5,31
3,-92,3,1,0,0,4,2,2011,18,1,3,1,1,0.46,-1908,5,1
4,-92,3,1,0,0,5,2,2011,18,1,3,1,1,0.46,-1907,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11054771,-107,2,0,9,2,1,10,2012,18,1,3,1,0,1.98,-1274,43,27
11054778,-105,2,0,9,2,1,10,2012,18,1,3,1,0,3.98,-1274,43,27
11054785,-104,2,0,9,2,1,10,2012,18,1,3,1,0,2.68,-1274,43,27
11054792,-103,2,0,9,2,1,10,2012,18,1,3,1,0,2.68,-1273,43,27


In [ ]:
# %%time
# seed = 99
# folds = 3
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import mean_squared_log_error
# import pickle

# skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)

# for idx, (train_index, test_index) in enumerate(skf.split(df_train.index,df_train['volume'])):
#     print(idx)
#     x_train = df_train.iloc[train_index].drop(['day','volume'], axis =1)
#     y_train = df_train.iloc[train_index]['volume']
#     x_valid = df_train.iloc[test_index].drop(['day','volume'], axis =1)
#     y_valid = df_train.iloc[test_index]['volume'] 
    
#     # Modeling
#     lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
#     lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
#     gbm = lgb.train(params, lgb_train,
# #                     num_boost_round=1000, 
#                     valid_sets=(lgb_train, lgb_eval),
#                     early_stopping_rounds= 50,#100,
#                     verbose_eval=200) #100)

#     pickle.dump(gbm,open( "20200501_model_%s_r2.pkl"%idx, "wb" ))
        
#     del gbm
#     gc.collect()
    

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[200]	training's rmse: 2.25134	valid_1's rmse: 2.30223
[400]	training's rmse: 2.17929	valid_1's rmse: 2.27837
[600]	training's rmse: 2.13866	valid_1's rmse: 2.269
Early stopping, best iteration is:
[618]	training's rmse: 2.13591	valid_1's rmse: 2.26865
1
Training until validation scores don't improve for 50 rounds
[200]	training's rmse: 2.25907	valid_1's rmse: 2.31285
[400]	training's rmse: 2.18477	valid_1's rmse: 2.27329
[600]	training's rmse: 2.14307	valid_1's rmse: 2.26023
[800]	training's rmse: 2.115	valid_1's rmse: 2.25519
[1000]	training's rmse: 2.09242	valid_1's rmse: 2.25177


In [ ]:
%%time
df_test['id'] = train_id
max_lag  = 120
trn_lst = 1913
for tdelta in range(1, 29):
    f_day = trn_lst+tdelta
    print(f_day)
    days = [f"d_{i}" for i in range(trn_lst-max_lag+tdelta,f_day+1)]
    tst = df_test[df_test['day'].isin(days)]

    print("rolling")
    tst['volume_1'] = tst[['id','volume']].groupby("id")['volume'].shift(1)
    tst['volume_2'] = tst[['id','volume']].groupby("id")['volume'].shift(2)
    tst['volume_3'] = tst[['id','volume']].groupby("id")['volume'].shift(3)
    
    tst['volume_7'] = tst[['id','volume']].groupby("id")['volume'].shift(7)
    tst['volume_28'] = tst[['id','volume']].groupby("id")['volume'].shift(28)
    
    print("mean")
    tst['rmean_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
    tst['rmean_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

    tst['rmean_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
    tst['rmean_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())
    
    print("std")
    tst['rstd_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).std())
    tst['rstd_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).std())
    tst['rstd_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).std())

    tst['rstd_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).std())
    tst['rstd_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).std())
    tst['rstd_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).std())

    print("max")
    tst['rmax_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).max())
    tst['rmax_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).max())
    tst['rmax_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).max())

    tst['rmax_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).max())
    tst['rmax_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).max())
    tst['rmax_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).max())

    print("min")
    tst['rmin_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).min())
    tst['rmin_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).min())
    tst['rmin_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).min())

    tst['rmin_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).min())
    tst['rmin_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).min())
    tst['rmin_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).min())

#     print("count")
#     tst['rcount_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
#     tst['rcount_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
#     tst['rcount_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

#     tst['rcount_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
#     tst['rcount_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
#     tst['rcount_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

    
    tst = tst[tst['day'] == "d_%s"%(f_day)]
    t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
    tst = tst.drop(['id','volume','day'], axis =1)
    
    # Crossvalidation 
    for idx in range(folds):
        gbm = pickle.load(open( "20200501_model_%s.pkl"%idx, "rb" ))
        df_test.loc[df_test.day=="d_%s"%(f_day),'volume'] += 1.028*gbm.predict(tst) / folds
    
    #0을 예측 못하니까 0으로 치환하는 것 -- test
#     df_test.loc[df_test.volume<0.5,'volume'] = 0
    
    


In [ ]:
cols = [f"d_{i}" for i in range(1914,1942)]

sub = df_test[df_test['day'].isin(cols)].loc[:,['id','volume']]
sub['F']= [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
sub = sub.set_index(["id", "F" ]).unstack()["volume"].reset_index()
sub.sort_values("id", inplace = True)
sub.reset_index(drop=True, inplace = True)                                                   
sub =sub[['id']+["F%s"% x for x in range(1,29)]]

sub = sub.fillna(0)

sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_20200501_1.csv",index=False)

In [ ]:
# Feature 체크하기
import pickle
gbm = pickle.load(open( "20200501_model_2.pkl", "rb" ))
# %%time

from sklearn.model_selection import train_test_split

def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))


x_train , x_valid = train_test_split(df_train, test_size =0.05, random_state = 99)
y_train, y_valid = x_train['volume'], x_valid['volume']

x_train = x_train.drop(['day','volume'], axis =1)
x_valid = x_valid.drop(['day','volume'], axis =1)


In [ ]:
%%time
pred = gbm.predict(x_valid)
print("original", rmse(y_valid, pred))
origin = rmse(y_valid, pred)

In [ ]:
%%time 
# 컬럼들을 랜덤하게 섞어서 예측해보는 것
# cols = list(x_valid.columns)

z = pd.read_csv('featuretest.csv')
cols = list(x for x in x_valid.columns if x not in list(z['columns']))
fin = dict()
for col in tqdm(cols):
    tmp = x_valid.copy()
    tmp[col] = np.random.permutation(tmp[col].values)
    t_rmse = rmse(y_valid, gbm.predict(tmp))
    gap = t_rmse-origin  # origin 보다 에러가 커졌으면 이 컬럼은 중요한 것 !
    print(col, t_rmse, gap)
    fin[col] = gap

print(fin)
new = pd.DataFrame()
new['columns'] =fin.keys()
new['values'] = fin.values()
z = pd.concat([z,new])
z.to_csv('featuretest.csv',index=False)
